Now begin testing the model.  Here are the functions we will be using:

In [1]:
##outputs the accuracy of the model
def accuracy(predictions, labels):
    preds = torch.argmax(predictions, dim=1)  
    return (preds == labels).float().mean().item()



#Testing Loop        
def test(bert, device, training_data, criterion, optimizer, epoch):
    # Set the model to evaluation mode
    bert.eval()


    for epoch in range(epochs):
        epoch_loss = 0.0  # To accumulate loss for this epoch
        testing_acc = 0.0  # Accumulate accuracy
        testing_loss = 0.0
    # Loop over the batches of data
    #loop over the batches of data:
        for i, batch in enumerate(testloader):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            inputs = {
                'input_ids': input_ids,
                'attention_mask': attention_mask
            }
        
        
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            
            #forward pass
            outputs = Bert_Classifier(**inputs)
            logits = outputs.pooler_output
        
            
            
            #calculate the loss
            loss = criterion(logits, labels) # Calculate the loss
            
            
            #Backward pass and optimize
            loss.backward() #compute the gradients
            optimizer.step() # Update the parameters
            
            
            # Accumulate loss for this batch
            epoch_loss += loss.item()
            
            #Print the statistics
            testing_loss += loss.item()
            testing_acc+= accuracy(logits, labels)
            if (i + 1) % 200 == 0: # print every two hundred batches
                print(f'Epoch {epoch}, Batch {i+1}, Loss: {testing_loss / 200:.4f}, Accuracy: {testing_acc / 200:.4f}')

Load the model and modules for testing.

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import random_split 
import torch.nn as nn
import torch.optim as optim
from transformers.models.bert.modeling_bert import BertModel


m_qa_inputs = torch.load("m_qa_inputs.pt", weights_only=False)
h_qa_inputs = torch.load("h_qa_inputs.pt", weights_only=False)
m_qa_tensors = m_qa_inputs['input_ids']
h_qa_tensors = h_qa_inputs['input_ids']

m_labels = torch.zeros(len(m_qa_tensors), dtype=torch.long)  # Middle school = 0
h_labels = torch.ones(len(h_qa_tensors), dtype=torch.long)   # High school = 1
labels = torch.cat([m_labels, h_labels], dim=0)

max_seq_len = max(m_qa_tensors.shape[1], h_qa_tensors.shape[1])
m_qa_tensors = torch.nn.functional.pad(m_qa_tensors, (0, max_seq_len - m_qa_tensors.shape[1]))
h_qa_tensors = torch.nn.functional.pad(h_qa_tensors, (0, max_seq_len - h_qa_tensors.shape[1]))
training_data = torch.cat([m_qa_tensors, h_qa_tensors], dim=0)

torch.load('Bert_Classifier.pt', weights_only=True)

Bert_Classifier = BertModel.from_pretrained("bert-base-uncased")
Bert_Classifier.load_state_dict(torch.load("Bert_Classifier.pt"), strict=False)
Bert_Classifier.eval()
dataset = TensorDataset(training_data, (training_data != 0).long(), labels) 
testloader = DataLoader(dataset, batch_size=32, shuffle=True )

Initiate the hyperparameters for the testing loop:

In [3]:
total_size = training_data.shape[0]
train_size = int(.6 * total_size)
val_size = int(.3 * total_size)
test_size = total_size - train_size - val_size

train_data, val_data, test_data = random_split(training_data, [train_size, val_size, test_size])

epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Bert_Classifier.parameters(), lr=1e-5, betas=(0.9, 0.999), weight_decay=1e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sequence_length = 200
batch_size = 64
input_dim = 500
d_model = 512


Move the model to the gpu

In [4]:
Bert_Classifier.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

Run the testing loop

In [5]:
test(Bert_Classifier, device, testloader, criterion, optimizer, epochs)

Epoch 0, Batch 200, Loss: 5.4650, Accuracy: 0.3620
Epoch 1, Batch 200, Loss: 4.7701, Accuracy: 0.6309
Epoch 2, Batch 200, Loss: 4.7020, Accuracy: 0.6402
Epoch 3, Batch 200, Loss: 4.6825, Accuracy: 0.6377
Epoch 4, Batch 200, Loss: 4.6741, Accuracy: 0.6403
